In [1]:
# import libraries
import pandas as pd
from ruqiya.ruqiya import clean_text
import re
import string
import nltk
import numpy as np
import ktrain
from ktrain import text
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
#remove emojies
import demoji

In [2]:
# read the dataset
delivCOMP =pd.read_csv('delivCOMP_Dataset.xlsx - all.csv')
delivCOMP.head()

,Tweet,Lable,Company
0,@CareemCare @lolliklulu الحين اذا العميل اتاخر...,Negative,Careem
1,@CareemCare اتواصلت معاكم بالتطبيق اناخذ مني م...,Negative,Careem
2,@gooole99 @CareemCare انا كمان مو شغال مدري اش...,Negative,Careem
3,@CareemCare @Cz10112 @Linakh32 ترسلون هنا على ...,Negative,Careem
4,@Li1Ra @HRDF_Care @CareemCare وش السالفه ليه معلق,Negative,Careem


In [3]:
delivCOMP.describe()

,Tweet,Lable,Company
count,5332,5356,5357
unique,4842,4,11
top,هنقرستيشن يملك أسوء خدمة عملاء,Negative,Hungrstation
freq,6,3914,1693


In [4]:
delivCOMP["Lable"].value_counts()

Negative     3914
Neutral       724
Positive      606
Negative      112
Name: Lable, dtype: int64

In [5]:
#show positive data
datasetpos = delivCOMP[delivCOMP.Lable == 'Positive']
datasetpos.shape

(606, 3)

In [6]:
#show negative data
datasetneg = delivCOMP[delivCOMP.Lable == 'Negative']
datasetneg.shape

(3914, 3)

In [7]:
#show neutral data
datasetneu = delivCOMP[delivCOMP.Lable == 'Neutral']
datasetneu.shape

(724, 3)

In [8]:
#sample of positive data
datasetpos = datasetpos.sample(frac=0.67,random_state=200)
datasetpos.shape

(406, 3)

In [9]:
# connect all above 
delivCOMP = pd.concat([datasetpos, datasetneg, datasetneu], axis=0)
delivCOMP.head()

,Tweet,Lable,Company
2104,@bebo55839907 @CareemCare كريم عروضهم حلوة واس...,Positive,Careem
1660,@JahezApp @fahadalattas49 @soomh_hfc @d82_s9 @...,Positive,Jahez
4785,@HungerStation طفرة عندي مليون ....\n شكرا لكم...,Positive,Hungrstation
3600,يالله حسيت بشعور حلو وانا أساعد وحده مضافه عند...,Positive,Mrsool
3525,@majood_1414 @HungerStation @AppMrsool بالنسبه...,Positive,Mrsool


In [10]:
#counts each lable
delivCOMP["Lable"].value_counts()

Negative    3914
Neutral      724
Positive     406
Name: Lable, dtype: int64

In [11]:
delivCOMP.isnull().any(axis=0)

Tweet       True
Lable      False
Company    False
dtype: bool

In [12]:
#Pre-Processing 
#clean and Normalize the text
def cleanTxt(text):
    text = re.sub(r'[A-Za-z0-9٠-٩]+', '', text) #Remove english letters,english NO.,Arabic NO. 
    text = demoji.replace(text, "")  #Remove Emojis
    return text
#Remove repeating character,punctuations,diacritics,stop words,Normalize Arabic
delivCOMP['Tweet'] = delivCOMP['Tweet'].astype(str).apply(clean_text)

delivCOMP['Tweet'] = delivCOMP['Tweet'].astype(str).apply(cleanTxt)
#Remove duplication tweets
delivCOMP.drop_duplicates(subset = 'Tweet', inplace = True)

# # #Tokenize the text 
# text = TweetTokenizer()
# delivCOMP['Tweet'] = delivCOMP['Tweet'].apply(text.tokenize)

In [13]:
delivCOMP

,Tweet,Lable,Company
2104,كريم عروضهم حلوه واسعارهم تقريبا المطاعم هنق...,Positive,Careem
1660,اله يبارك فيكم شكرا شريكنا الميز...,Positive,Jahez
4785,طفره عندي \nشكرا تجاوب جميل \nبالتوفيق لكم,Positive,Hungrstation
3600,ياله حسيت بشعور حلو اساعد وحده مضافه عندي بالس...,Positive,Mrsool
3525,بالنسبه لعميل افضل وبذات خدمه العملاء\nطبع...,Positive,Mrsool
...,...,...,...
5332,برجاء فتح حسابي تويو,Neutral,ToYou
5333,تويومودكم تفوزوني وتقولون بالعافيه,Neutral,ToYou
5334,\nكود تويو \n طلب ريال رصيد,Neutral,ToYou
5348,الحظات اهم اتصال تويو,Neutral,ToYou


In [14]:
# print sample of dataset 
delivCOMP["Tweet"].sample(20)

2822     تقولون  طلبات توصيل بلاش بدون كود  عمري طلبت ...
4596     مندوب توصيل سحبتو الفلوس المحفظه وقلتو نزلتوه...
2767     حسبي اله ونعم الوكيل ساعه وزياده عشان المندوب...
3922                                           الغي الطلب
1468                         المندوب وصل الطلب غلط ورف...
4049     لاسف سياسيه التطبيق الكابتن تحويل الاموال وجو...
3993     دايم اقولها بصراحه يرحم والديكم ترا مندوبينكم...
1215     السلام عليكم  الطلب وصلني بدون خبز وحاولت الا...
5352                              تويو طالبه ساعه وماوصل 
3915     طيب وش الحل النصابين\nيطلبون الطلب ويقول حط ا...
4425      معاك لاسف  صاير فاشل  كلمتهم خاص ارسلوك لدعم...
933      اكتشفت جاهز سيء  يستطيع حل مشكله تواجه العميل...
1899        طلبت عندكم فرع الرس هنقرستيشن وصلني طلبي ناقص
4842     طلب طالبه ليا ساعه ونص واتواصل معكم يتكنسل وا...
3292     ياخي مندوبك تاخر علينا ساعتين\nوفوق جاينا الط...
4696     شيكت ساعتين افر ولاعرض رخيص وعروض مطاعم مابيه...
1201           توني طلبت عندكم وصلني الطلب مفتوح بهالشكل 
1738          

In [15]:
#split the dataset to train - test
msk = np.random.rand(len(delivCOMP)) < 0.8
df_train = delivCOMP[msk]
df_test = delivCOMP[~msk]
#print train shape
df_train.shape

(3682, 3)

In [16]:
#print test shape
df_test.shape

(847, 3)

In [17]:
#counts each lable for test dataset
df_test.Lable.value_counts()

Negative    659
Neutral     123
Positive     65
Name: Lable, dtype: int64

In [18]:
#counts each lable for test dataset
df_train.Lable.value_counts()

Negative    2833
Neutral      538
Positive     311
Name: Lable, dtype: int64

In [19]:
# set hyperparameters
maxlen = 64
batch_size = 16
lr = 2e-5 
epochs = 3

In [20]:
#arabert modelPre-trained Transformers for the Arabic text
MODEL_NAME = 'aubmindlab/bert-base-arabertv01'
t = text.Transformer(MODEL_NAME, maxlen=maxlen)

In [21]:
trn = t.preprocess_train(df_train.Tweet.values, df_train.Lable.values)
tst = t.preprocess_test(df_test.Tweet.values, df_test.Lable.values)

preprocessing train...
language: ar
train sequence lengths:
	mean : 12
	95percentile : 27
	99percentile : 35


Is Multi-Label? False
preprocessing test...
language: ar
test sequence lengths:
	mean : 12
	95percentile : 25
	99percentile : 36


In [22]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=tst, batch_size=batch_size)

In [23]:
history = learner.fit_onecycle(lr, epochs)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
231/231 [==============================] - 1383s 6s/step - loss: 0.5798 - accuracy: 0.7868 - val_loss: 0.4425 - val_accuracy: 0.8501
Epoch 2/3
231/231 [==============================] - 1555s 7s/step - loss: 0.4029 - accuracy: 0.8561 - val_loss: 0.3466 - val_accuracy: 0.8642
Epoch 3/3
231/231 [==============================] - 2240s 10s/step - loss: 0.2385 - accuracy: 0.9188 - val_loss: 0.3532 - val_accuracy: 0.8654


In [24]:
learner.validate(val_data=tst) # summery

27/27 [==============================] - 82s 3s/step
              precision    recall  f1-score   support

           0       0.90      0.95      0.93       659
           1       0.65      0.50      0.57       123
           2       0.78      0.71      0.74        65

    accuracy                           0.87       847
   macro avg       0.78      0.72      0.74       847
weighted avg       0.86      0.87      0.86       847



array([[625,  29,   5],
       [ 53,  62,   8],
       [ 14,   5,  46]])

In [25]:
p = ktrain.get_predictor(learner.model, t)

In [26]:
p.predict("الهلال افضل فريق كورة")

1/1 [==============================] - 0s 107ms/step


'Positive'

In [27]:
p.predict("هنقرستيشن افضل تطبيق توصيل")

1/1 [==============================] - 0s 97ms/step


'Positive'

In [28]:
p.predict("مرسول اسوأ تطبيق توصيل")

1/1 [==============================] - 0s 99ms/step


'Negative'

In [29]:
p.predict("جاهز رجعوا فلوسي")

1/1 [==============================] - 0s 99ms/step


'Negative'

In [30]:
p.predict("شكرا لكم")

1/1 [==============================] - 0s 86ms/step


'Positive'

In [31]:
p.predict("هنقرستيشن حرامية")

1/1 [==============================] - 0s 89ms/step


'Negative'

In [32]:
p.predict("ردوا على الخاص")

1/1 [==============================] - 0s 84ms/step


'Negative'

In [33]:
p.predict("المندوب تأخر ")

1/1 [==============================] - 0s 84ms/step


'Negative'

In [46]:
p.predict("المندوب تأخر جدا ")

1/1 [==============================] - 0s 397ms/step


'Negative'

In [50]:
from tkinter import *
# Top level window
frame = tk.Tk()
frame.title("TextBox Input")
frame.geometry('400x200')

top.geometry("500x300")

def printInput():
	inp = inputtxt.get(1.0, "end-1c")
	lbl.config(text = "Provided Input: "+p.predict(str(inp))


# TextBox Creation
inputtxt = tk.Text(frame,
				height = 5,
				width = 20)

inputtxt.pack()
 

# Button Creation
printButton = tk.Button(frame,
						text = "Print",
						command = printInput)
printButton.pack()

# Label Creation
lbl = tk.Label(frame, text = "")
lbl.pack()
frame.mainloop()




SyntaxError: invalid syntax (2093024014.py, line 15)